In [1]:
import tensorflow as tf
import numpy as np
import datetime

In [2]:
tf.keras.backend.clear_session()

input_shape = (368, 368, 3)  #only the input shape is shared between the models
#transfer learning model definition
input_layer_vgg = tf.keras.layers.Input(shape=input_shape)
vgg_input_model = tf.keras.applications.VGG16(weights='imagenet', include_top=False, input_tensor=input_layer_vgg)
name_last_layer = "block3_pool" #the last layer to copy

tf.keras.backend.clear_session() #clean the graph from the transfer learning model

input_layer = tf.keras.layers.Input(shape=input_shape) #define the input layer for the first model
x=input_layer
for layer in vgg_input_model.layers[1:]:    #copy over layers, without the other input layer
    config=layer.get_config()  #get config
    weights=layer.get_weights() #get weights
    #print(config)
    copy_layer=type(layer).from_config(config) #create the new layer from config
    x=copy_layer(x) #connect to previous layers,
                    #required for the proper sizing of the layer,
                    #set_weights will not work without it
    copy_layer.set_weights(weights)    
    
    if layer.name == name_last_layer:
        break
del vgg_input_model
        
input_model=tf.keras.Model(inputs=input_layer,outputs=x) #create the new model,
                                                        #if needed x can be used further doen the line

{'bias_regularizer': None, 'filters': 64, 'bias_constraint': None, 'kernel_regularizer': None, 'activation': 'relu', 'use_bias': True, 'kernel_initializer': {'class_name': 'GlorotUniform', 'config': {'seed': None}}, 'kernel_size': (3, 3), 'kernel_constraint': None, 'trainable': True, 'padding': 'same', 'name': 'block1_conv1', 'dilation_rate': (1, 1), 'activity_regularizer': None, 'dtype': 'float32', 'strides': (1, 1), 'data_format': 'channels_last', 'bias_initializer': {'class_name': 'Zeros', 'config': {}}}
{'bias_regularizer': None, 'filters': 64, 'bias_constraint': None, 'kernel_regularizer': None, 'activation': 'relu', 'use_bias': True, 'kernel_initializer': {'class_name': 'GlorotUniform', 'config': {'seed': None}}, 'kernel_size': (3, 3), 'kernel_constraint': None, 'trainable': True, 'padding': 'same', 'name': 'block1_conv2', 'dilation_rate': (1, 1), 'activity_regularizer': None, 'dtype': 'float32', 'strides': (1, 1), 'data_format': 'channels_last', 'bias_initializer': {'class_name'

In [3]:
# layer = vgg_input_model.layers[1]
# config=layer.get_config()
# weights=layer.get_weights()
# print(config)
# copy_layer=type(layer).from_config(config)
# #copy_layer.set_weights(weights)


In [4]:
input_model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 368, 368, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 368, 368, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 368, 368, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 184, 184, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 184, 184, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 184, 184, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 92, 92, 128)       0     

In [5]:
inp=np.random.random((10,368,368,3))
out=np.random.random((10,46,46,256))

In [6]:
tensorboard_dir='gs://dl_training_results/tensorboard/test'+datetime.datetime.now().strftime("%a%d%m%y-%H%M")
tensorboard_callback=tf.keras.callbacks.TensorBoard(  log_dir=tensorboard_dir)

In [7]:
@tf.function
def mse_2d_loss(y_true, y_pred):
    pixel_losses=tf.keras.losses.mean_squared_error(y_true, y_pred)
    return tf.math.reduce_mean(pixel_losses,axis=-1)

In [8]:
input_model.compile(optimizer=tf.keras.optimizers.Adam()
                ,loss=mse_2d_loss
                ,metrics=['acc']                    
               )

input_model.fit(
    inp,out,epochs=2
    
    ,callbacks=[tensorboard_callback]
)

Train on 10 samples
Epoch 1/2
10/10 [==============================] - 18s 2s/sample - loss: 83.6260 - acc: 0.0041
Epoch 2/2
10/10 [==============================] - 19s 2s/sample - loss: 32.3674 - acc: 0.0029


In [9]:
vgg_input_model.layers

NameError: name 'vgg_input_model' is not defined

In [ ]:
# input_tensor = tf.keras.layers.Input(shape=input_shape)
# vgg_input_model = tf.keras.applications.VGG16(weights='imagenet', include_top=False)

# final_vgg_kayer = vgg_input_model.get_layer("block3_pool")
# input_layer=input_model.get_layer("block1_conv1")
# input_model = tf.keras.Model(inputs=input_layer.inputs, outputs=final_vgg_kayer.output)

In [ ]:
# name_last_layer = "block3_pool"

# model2 = tf.keras.Sequential()
# for layer in vgg_input_model.layers:
#     print(str(layer.name))
#     if layer.name == name_last_layer:
#         break
#     model2.add(layer)

In [ ]:
!gsutil -m rm -r gs://dl_training_results/tensorboard/*

In [ ]:
l=tf.keras.layers.Conv2D(128,3)


In [ ]:
l(input_tensor)

In [ ]:
l_cfg=l.get_config()
l_wts=l.get_weights()

In [ ]:
l_wts